In [317]:
import torch  
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import time

"Using Cuda" if torch.cuda.is_available() else "Not using Cuda"

'Using Cuda'

# Loss
Suppose we have data that consists of independent vector $x_i$ and dependent $y_i$ (where $i$ is the $i$-th value in the dataset)

Suppose i have $\vec{p} = [1,3,5,2]$ of length $N$, I want to know the corresponding matching set of numbers $\vec{q} = [q_1, q_2, \ldots q_n]$ such that the following function is minimized:

$$H(\vec{p}, \vec{q}) = -\sum\limits^N_{i=1} p_i \ln(q_i)$$

with the $C$ constraint that $C := (\sum\limits_{i=1} p_i = \sum\limits_{i=1} q_i)$

So bellow lets try some examples

In [318]:
p = np.array([5,1,4,6,2,4])
qs = [p, np.array([3,7,1,4,1,6]), p * 2, p * 0.5]
h = lambda p,q : -np.sum(p * np.log(q))

for q in qs:
    print(h(p,q))

-31.27439562761785
-22.923775636027425
-46.523633599936645
-16.02515765529905


Well the solution to minimize $H(\vec{p},\vec{q})$ with the constraint $C$ is when $\vec{p}=\vec{q}$. You can prove this fact via Lagrange multipliers. 

This also works if we want to minimize $\int^{-\infty}_{\infty} p(x)ln(q(x)) dx$ with $\int^{-\infty}_{\infty} p(x) dx = \int^{-\infty}_{\infty} q(x) dx$

This why the function represents "cross" - entropy as it measuring the difference of a measure in function $q$ with respect to a known function $p$ 

## Cross Entropy Loss 

In classification problems we have an input image called $x$. The image needs to be into buckets (car, dog, mountain, person ... etc). We will call the true likelihood of an image $x$ belonging to class $i$ as $p_i$. The goal of a classifier is to create a function $f$ such that $$f(x)=\vec{q}$$

Where $\vec{q}$ is as close to $\vec{p}$ as possible.  

Note 

1. $\vec{p}$ and $\vec{q}$ are probability mass functions hence $\sum p_i = \sum q_i = 1$ (hence the constraint earlier)
1. We know the class $c_i$ that $x$ belongs to and so $p_{c_i}= 1$ and $\forall j \not = i (p_{c_j} = 0)$ and so we want $f(x)_{c_j} \approx 1$

So to minimize the difference between $p$ (our vector of actual likelihoods) and $q$ our model, $f$'s prediction of $q$ we can minimize the loss function. 

$H(p,q)=-\sum^N_i p_i \ln(q_i)$

Since the minimum occurs when $\forall i \in \set{1,\ldots, n} (q_i = p_i)$ so if we know the class $i$ that image belongings to then $$H(p,q)=-ln(q_{c_i})$$ 

In [319]:
q = np.random.rand(10)
q = q / q.sum()

p = np.zeros(10, dtype=np.int32) # for example 10 classes where the 5th class is the correct one
p[4] = 1

p

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0])

In [320]:
h = - np.log(q[p>0])
h

array([1.83269366])

## For multiple images
Say we have $n$ images $x_1, \ldots, x_n$ with the true class labeled $C(j)$ for the $j$-th image $x_j$. Then the probability that we predict the $j$-th image belongs to some class $c_j$ is $q_n(c_j)$. 

- Thus the predicted probability of the image $x_j$ belonging to its true class $C(j)$ is $q_j(C(j))$

Now we sum it together 
$$\mathcal{L} (p,q) = \sum^N_{i=1} H(p_i, q_i) = - \sum^N_{i=1} \ln(q_i(C(i)))$$


In [321]:
number_of_classes = 10
number_of_images = 4 

In [322]:
q = np.random.rand(number_of_images*number_of_classes).reshape(number_of_images,number_of_classes)
q = q / q.sum(axis=1)[:,None]
q

array([[0.16104308, 0.00160313, 0.09912951, 0.11877473, 0.02085183,
        0.11888477, 0.12304694, 0.106756  , 0.10343718, 0.14647281],
       [0.11829841, 0.08343518, 0.12314581, 0.06598963, 0.08475161,
        0.0875063 , 0.07754829, 0.12775287, 0.10002658, 0.13154531],
       [0.1826857 , 0.1650255 , 0.0581281 , 0.01421221, 0.07737811,
        0.03450343, 0.21196609, 0.02686344, 0.06547849, 0.16375893],
       [0.02440514, 0.0046603 , 0.1383402 , 0.1697049 , 0.17852368,
        0.09615656, 0.07641861, 0.06858985, 0.19147401, 0.05172675]])

In [323]:
p = np.zeros((number_of_images,number_of_classes), dtype=np.int32)
p[0][4] = 1
p[1][2] = 1
p[2][8] = 1
p[3][6] = 1
p

array([[0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]])

In [324]:
def loss(q,p):
    hs = - np.log(q[p>0])
    l = hs.sum()
    return l

loss(q,p)

11.262262285207969

Now lets say some NN outputs $\hat{y}_i$ for image $x_i$ that now needs normalized be normalized to get the output $q_i$. In other words $\mathrm{normalize}(\hat{y}_i)=q_i$. And we can define $\mathrm{normalize}$ a few ways but most often we need it to be related to rules of a PDF i.e. that is 
1. In range $[0,1]$
1. Closer some dimension $i$ is to $0$, the less confident that its class $c_i$ 
1. Closer some dimension $i$ is to $1$, the more confident that its class $c_i$ 
1. $\sum^{n}_{i=1}q_i = 1$ Per image 


The equation we use is 


$$\mathrm{normalize}(\hat{y}_{i_j}) = \dfrac{\exp(\hat{y}_{i_j})}{\sum^{n}_{k=1} \exp(\hat{y}_{i_k})}$$

So our loss is 


$$
L(\hat{y}) = - \sum^{m}_{i=1} 
\ln(
\dfrac
{\exp(\hat{y}_{i_{C(i)}})}
{\sum^n_{j=1} \exp(\hat{y_{i_j}})}
)
$$



In [325]:
# lets say our NN spat out y_hat like so 
y_hat = np.random.rand(number_of_images*number_of_classes).reshape(number_of_images,number_of_classes)
def normalize(y_hat):
    a = np.exp(y_hat)
    b = np.sum(np.exp(y_hat), axis=1).reshape(number_of_images,1)
    return a/b

q = normalize(y_hat)
q

array([[0.0890691 , 0.14254244, 0.07655385, 0.13783901, 0.08819724,
        0.11909426, 0.1232031 , 0.06991312, 0.06991951, 0.08366837],
       [0.0768645 , 0.0733495 , 0.07357584, 0.06452405, 0.13638038,
        0.07959567, 0.13188526, 0.12443219, 0.12605999, 0.11333262],
       [0.07643283, 0.07297882, 0.05921184, 0.12551416, 0.13398697,
        0.10120047, 0.12873027, 0.0845045 , 0.13866755, 0.07877258],
       [0.07158362, 0.11993187, 0.09323131, 0.08643524, 0.10012291,
        0.14565543, 0.08731729, 0.05900629, 0.13901902, 0.09769702]])

In [326]:
# proof it works
q.sum(axis=1)

array([1., 1., 1., 1.])

In [327]:
c = np.where(p) # get the indexes of the correct classes
c


(array([0, 1, 2, 3], dtype=int64), array([4, 2, 8, 6], dtype=int64))

In [328]:
hs = -np.log(q[c])
loss = hs.sum()
loss

9.451500978121764